In [1]:
import cv2
import numpy as np


In [2]:
image = cv2.imread("D:\\Dat_Hoctap\\2023\\Tamper_Recover\\Data\\bmp\\lena.bmp")

In [11]:
def check_one_block(B,th):
    diff_value = np.zeros((2,2),dtype=np.int16)
    avg_B = np.mean(B).round().astype(np.uint8)
    bits_B =  np.unpackbits(avg_B)
    bits_B[6] = 0
    bits_B[7] = 0
    avg_B = np.packbits(bits_B)
    is_smooth = True
    
    for i in range(2):
        for j in range(2):
            bits = np.unpackbits(B[i,j])
            bits[6] = 0
            bits[7] = 0
            e_value = np.packbits(bits)
            diff_value[i,j] = int(e_value) - int(avg_B)
            if np.abs(diff_value[i,j]) > th:
                is_smooth = False

    return diff_value,is_smooth,avg_B


In [19]:
def smooth_infor(array,th):
    nr,nc = array.shape
    nr_table = int(nr/2)
    nc_table = int(nc/2)
    avg_image = np.zeros((nr_table,nc_table),dtype=np.uint8)
    n_smooth_b = 0
    diff_value = np.zeros((nr,nc),dtype=np.int16)
    diff_non_smooth = []
    for ir in range(nr_table):
        for ic in range(nc_table):
            B = array[ir*2:ir*2+2,ic*2:ic*2+2]
            diff_value[ir*2:ir*2+2,ic*2:ic*2+2],is_smooth = check_one_block(B,th)
            if is_smooth : n_smooth_b += 1
            else:
                diff_non_smooth.append(diff_value[ir*2:ir*2+2,ic*2:ic*2+2])
    return diff_value,n_smooth_b,diff_non_smooth



In [20]:
diff_image = np.zeros_like(image,dtype=np.int16)
n_smooth_b = np.zeros(3)
diff_non_smooth = []
for i in range(3):
    diff_image[:,:,i],n_smooth_b[i],tmp = smooth_infor(image[:,:,i],th)
    diff_non_smooth.append(tmp)

In [21]:
first = np.array(diff_non_smooth[0])

In [24]:
print(first.min())
print(first.max())

-76
76


In [17]:
print(256*256 - n_smooth_b)
print(256*256)

[4883. 5973. 3826.]
65536
